In [8]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///chinook.db')
cur = engine.connect()

In [9]:
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [16]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

In [10]:
print("Available tables:", metadata.tables.keys())

Available tables: dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'tracks', 'media_types', 'playlist_track', 'playlists', 'sqlite_sequence', 'sqlite_stat1'])


In [17]:
from sqlalchemy import select 
tracks = Base.classes['tracks']

# using expressions
query = select([tracks]).limit(3)
display_results(query)

C:\Users\olgak\anaconda3\lib\site-packages\pandas\io\sql.py:1402: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return self.connectable.execution_options().execute(*args, **kwargs)


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99



SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



In [22]:
output = cur.execute("SELECT Name, AlbumId FROM tracks")
output.fetchmany(20)

[('For Those About To Rock (We Salute You)', 1),
 ('Balls to the Wall', 2),
 ('Fast As a Shark', 3),
 ('Restless and Wild', 3),
 ('Princess of the Dawn', 3),
 ('Put The Finger On You', 1),
 ("Let's Get It Up", 1),
 ('Inject The Venom', 1),
 ('Snowballed', 1),
 ('Evil Walks', 1),
 ('C.O.D.', 1),
 ('Breaking The Rules', 1),
 ('Night Of The Long Knives', 1),
 ('Spellbound', 1),
 ('Go Down', 4),
 ('Dog Eat Dog', 4),
 ('Let There Be Rock', 4),
 ('Bad Boy Boogie', 4),
 ('Problem Child', 4),
 ('Overdose', 4)]

print out the first 10 track sales from the invoice_items table
- for these first 10 sales, print what are the names of the track sold, and the quantity sold

In [26]:
query = 'SELECT Name, Quantity  from invoice_items as i join tracks as t on i.TrackId=t.TrackId '
result_df = pd.read_sql(query, engine)
result_df.head(10)

,Name,Quantity
0,Balls to the Wall,1
1,Restless and Wild,1
2,Put The Finger On You,1
3,Inject The Venom,1
4,Evil Walks,1
5,Breaking The Rules,1
6,Dog Eat Dog,1
7,Overdose,1
8,Love In An Elevator,1
9,Janie's Got A Gun,1


In [27]:
query = 'SELECT * from invoice_items'
result_df = pd.read_sql(query, engine)
result_df.head(10)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
5,6,2,12,0.99,1
6,7,3,16,0.99,1
7,8,3,20,0.99,1
8,9,3,24,0.99,1
9,10,3,28,0.99,1


 print the names of top 10 tracks sold, and how many they times they were sold

In [32]:
query = 'SELECT Name, sum(Quantity) as Quantity  from invoice_items as i join tracks as t on i.TrackId=t.TrackId Group By Name Order By Quantity DESC'
result_df = pd.read_sql(query, engine)
result_df.head(10)

,Name,Quantity
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3


Who are the top 10 highest selling artists?

In [34]:
query = 'SELECT * FROM artists'
result_df = pd.read_sql(query, engine)
result_df.head(10)

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
5,6,Antônio Carlos Jobim
6,7,Apocalyptica
7,8,Audioslave
8,9,BackBeat
9,10,Billy Cobham


In [35]:
query = 'SELECT * FROM albums'
result_df = pd.read_sql(query, engine)
result_df.head(10)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,9,Plays Metallica By Four Cellos,7
9,10,Audioslave,8


In [37]:
query = 'SELECT a.Name, sum(i.Quantity) as Quantity  from artists as a join albums as alb on a.ArtistId=alb.ArtistId JOIN tracks as t ON t.AlbumId=alb.AlbumId JOIN invoice_items as i on i.TrackId=t.TrackId  Group By Name Order By Quantity DESC'
result_df = pd.read_sql(query, engine)
result_df.head(10)

OperationalError: (sqlite3.OperationalError) ambiguous column name: Name
[SQL: SELECT a.Name, sum(i.Quantity) as Quantity  from artists as a join albums as alb on a.ArtistId=alb.ArtistId JOIN tracks as t ON t.AlbumId=alb.AlbumId JOIN invoice_items as i on i.TrackId=t.TrackId  Group By Name Order By Quantity DESC]
(Background on this error at: https://sqlalche.me/e/14/e3q8)